# Update Google Calendar From Excel File

### Import & Setup

In [ ]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from httplib2 import Http

In [ ]:
scopes = ['https://www.googleapis.com/auth/calendar']

In [ ]:
flow = InstalledAppFlow.from_client_secrets_file("client_secret_calendar.json", scopes=scopes)
credentials = flow.run_console()

#log in and copy credentials


In [ ]:
import pickle
pickle.dump(credentials, open("token.pkl", "wb")) 
credentials = pickle.load(open("token.pkl", "rb"))

In [ ]:
service = build("calendar", "v3", credentials=credentials,cache_discovery=False)

In [ ]:
result = service.calendarList().list().execute()

In [ ]:
# The calendar you want to update may be a different index
calendar_id = result['items'][1]['id']

In [ ]:
result = service.events().list(calendarId=calendar_id).execute()
print(result['items'][0])

### Read In Excel Source File

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

file_path = 'C:/Users/Jeremy_Freelance/Desktop/Plan.xlsx'
df_calendar = pd.read_excel(file_path)

# df_calendar.head()
date = df_calendar['Date']
general_bucket = df_calendar['General Bucket']
start_time = df_calendar['Start Date Time']
end_time = df_calendar['End Date Time']
task = df_calendar['Task']

### Function To Create Event

In [ ]:
def create_event(start_date_time, end_date_time, event_title):
    description=None
    location=None
    
    event = {
        'summary': event_title,
        'location': location,
        'description': description,
        'start': {
            'dateTime': start_date_time,
            'timeZone': 'America/New_York',
        },
        'end': {
            'dateTime': end_date_time,
            'timeZone': 'America/New_York',
        },
    }
        
    return service.events().insert(calendarId=calendar_id, body=event,sendNotifications=True).execute()

### We Will Now Loop Through 1 Row Of The Dataframe At A Time

In [ ]:
import json
df_test=df_calendar[['Start Date Time','End Date Time','Task']]
df_test['Start Date Time'] = df_test['Start Date Time'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))
df_test['End Date Time'] = df_test['End Date Time'].apply(lambda x: x.strftime("%Y-%m-%dT%H:%M:%S"))

start = df_test['Start Date Time']
end = df_test['End Date Time']
title = df_test['Task']

In [ ]:
for i in range(len(df_test)) : 
    create_event(start[i],end[i],title[i])
print("Completed")